Rahmanda Afebrio Yuris Soesatyo - Chapter 11:,Training Deep Neural Networks

1. Vanishing dan Exploding Gradients serta Weight Initialization

Pada deep neural networks, proses pelatihan sering mengalami kendala karena nilai gradien dapat:

Menghilang (vanishing gradients), yaitu gradien menjadi sangat kecil

Membesar berlebihan (exploding gradients), yaitu gradien tumbuh tidak terkendali

Masalah ini muncul saat gradien dipropagasikan mundur melalui banyak lapisan selama proses backpropagation.

Kondisi tersebut menjadi semakin parah ketika menggunakan fungsi aktivasi klasik seperti sigmoid atau tanh, terutama jika dikombinasikan dengan inisialisasi bobot yang kurang tepat. Akibatnya, lapisan-lapisan awal hampir tidak menerima sinyal pembelajaran, sehingga gagal belajar secara efektif.

Weight Initialization

Untuk mengatasi permasalahan ini, Glorot dan Bengio memperkenalkan Xavier (Glorot) Initialization.
Pada pendekatan ini, bobot diinisialisasi secara acak dengan varians tertentu sehingga:

Varians output setiap layer relatif sama dengan varians input

Sinyal aktivasi dan gradien tetap stabil di seluruh jaringan

Skema ini terbukti membantu mengurangi risiko vanishing maupun exploding gradients, khususnya pada tahap awal pelatihan jaringan.

Skema Inisialisasi Berdasarkan Fungsi Aktivasi

Setiap fungsi aktivasi memiliki karakteristik yang berbeda, sehingga membutuhkan metode inisialisasi bobot yang sesuai agar performa optimal. Beberapa skema yang umum digunakan antara lain:

Glorot (Xavier) Initialization
Cocok untuk fungsi aktivasi tanh, sigmoid, dan softmax

He Initialization
Direkomendasikan untuk ReLU dan variannya

LeCun Initialization
Digunakan untuk fungsi aktivasi SELU

Parameter serta perbandingan antar skema inisialisasi ini biasanya dirangkum dalam literatur pembelajaran mendalam, seperti yang ditunjukkan pada Tabel 11-1.

In [11]:
from tensorflow import keras

# He initialization (untuk ReLU dan variannya)
he_layer = keras.layers.Dense(
    10, activation="relu", kernel_initializer="he_normal"
)

# Glorot (default Keras) dengan VarianceScaling
he_avg_init = keras.initializers.VarianceScaling(
    scale=2., mode="fan_avg", distribution="uniform"
)
glorot_like_layer = keras.layers.Dense(
    10, activation="sigmoid", kernel_initializer=he_avg_init
)

# LeCun initialization (untuk SELU)
lecun_layer = keras.layers.Dense(
    10, activation="selu", kernel_initializer="lecun_normal"
)


2. Advanced Activation Functions (ReLU, ELU, SELU)

Fungsi aktivasi klasik seperti sigmoid cenderung mengalami saturasi, yaitu kondisi di mana gradien mendekati nol untuk nilai input yang besar. Masalah ini memperparah fenomena vanishing gradients, terutama pada jaringan saraf yang dalam.

Penelitian lanjutan menunjukkan bahwa fungsi aktivasi non-saturating, seperti ReLU, jauh lebih stabil dan efisien ketika digunakan pada deep neural networks, sehingga mempercepat konvergensi dan meningkatkan performa pelatihan.

Variants of ReLU

Untuk mengatasi kelemahan ReLU standar—khususnya masalah dead neurons—berbagai varian ReLU dikembangkan, antara lain:

Leaky ReLU
Mempertahankan gradien kecil namun tidak nol pada sisi input negatif, sehingga neuron tetap dapat belajar meskipun menerima nilai negatif.

Randomized ReLU (RReLU)
Menggunakan kemiringan acak pada sisi negatif selama proses training. Pendekatan ini juga berperan sebagai bentuk regularisasi.

Parametric ReLU (PReLU)
Menjadikan kemiringan pada sisi negatif sebagai parameter yang dapat dipelajari langsung oleh model, sehingga memberikan fleksibilitas lebih dalam proses pembelajaran.

In [12]:
from tensorflow import keras

# Leaky ReLU sesudah Dense
model = keras.models.Sequential([
    keras.layers.Dense(10, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(alpha=0.2),
])

# PReLU (alpha dipelajari)
model_prelu = keras.models.Sequential([
    keras.layers.Dense(10, kernel_initializer="he_normal"),
    keras.layers.PReLU(),
])

# SELU + LeCun normal untuk self-normalizing dense net
selu_layer = keras.layers.Dense(
    10, activation="selu", kernel_initializer="lecun_normal"
)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


3. Batch Normalization & Gradient Clipping
Batch Normalization (BN)

Batch Normalization adalah teknik yang menormalkan input pada suatu layer dengan cara:

melakukan zero-centering dan scaling berdasarkan statistik mini-batch,

kemudian menerapkan transformasi ulang menggunakan parameter terlatih γ (gamma) dan β (beta).

Manfaat Batch Normalization antara lain:

mempercepat proses konvergensi training,

mengurangi sensitivitas terhadap inisialisasi bobot,

berperan sebagai regularisasi implisit yang membantu mencegah overfitting.

Penempatan Batch Normalization

Batch Normalization umumnya ditempatkan:

sebelum fungsi aktivasi pada hidden layer, atau

setelah fungsi aktivasi, tergantung arsitektur dan implementasi.

Jika digunakan pada layer pertama, Batch Normalization bahkan dapat menggantikan proses standardization pada data input.

Gradient Clipping

Gradient clipping digunakan untuk mengatasi masalah exploding gradients, terutama pada model yang sangat dalam seperti Recurrent Neural Networks (RNN).

Teknik ini membatasi besar gradien dengan dua pendekatan:

Value clipping, yaitu membatasi setiap elemen gradien secara individual.

Norm clipping, yaitu membatasi norma total gradien agar tidak melebihi threshold tertentu.

Gradient clipping membantu menjaga stabilitas training dan mencegah pembaruan bobot yang terlalu ekstrem.

In [15]:
from tensorflow import keras

# Model dengan Batch Normalization setelah setiap hidden layer
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

# Optimizer dengan gradient clipping by value
optimizer = keras.optimizers.SGD(learning_rate=0.01, clipvalue=1.0)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

4. Transfer Learning & Unsupervised Pretraining
Untuk deep neural network berukuran besar, melatih model dari awal (training from scratch) sering kali tidak efisien, terutama ketika data berlabel terbatas. Transfer learning menawarkan solusi dengan memanfaatkan model yang telah dilatih sebelumnya pada tugas atau domain yang serupa.

Pada pendekatan ini, layer bawah jaringan (yang berfungsi sebagai feature extractor) digunakan kembali, sementara output layer diganti dan dilatih ulang agar sesuai dengan tugas baru.

Practical Workflow
Dalam praktik, proses transfer learning biasanya dilakukan sebagai berikut:

Reuse layer bawah dari model pra-latih.
Bekukan (freeze) layer yang digunakan kembali agar bobot awal tidak langsung rusak oleh gradien besar dari layer baru.
Latih output layer baru selama beberapa epoch.
Unfreeze sebagian upper layers dan lakukan fine-tuning dengan learning rate yang lebih kecil.
Strategi ini menjaga stabilitas pembelajaran sekaligus memungkinkan adaptasi bertahap terhadap tugas baru.

Unsupervised and Self-Supervised Pretraining
Jika tidak tersedia model pra-latih yang sesuai, dapat digunakan unsupervised pretraining atau self-supervised learning untuk mengekstraksi representasi fitur yang berguna, misalnya melalui:

Autoencoders
Generative Adversarial Networks (GANs)
Tugas bantu (auxiliary tasks) berbasis self-supervision
Layer bawah hasil pretraining ini kemudian digunakan kembali untuk tugas utama yang berlabel.

In [ ]:
from tensorflow import keras

# Load model A (sudah dilatih)
model_A = keras.models.load_model("my_model_A.h5")

# Reuse semua layer kecuali output, lalu tambah output baru biner
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

# Bekukan layer lama
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer="sgd",
                     metrics=["accuracy"])

# Warm-up training
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                           validation_data=(X_valid_B, y_valid_B))

# Unfreeze dan fine-tune dengan learning rate kecil
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

optimizer = keras.optimizers.SGD(learning_rate=1e-4)
model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=optimizer,
                     metrics=["accuracy"])

history_fine = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                                validation_data=(X_valid_B, y_valid_B))


5. Faster Optimizers & Learning Rate Scheduling

Selain desain arsitektur jaringan, efektivitas proses pelatihan model deep learning sangat dipengaruhi oleh pemilihan optimizer dan strategi learning rate scheduling. Optimizer yang tepat mampu mempercepat proses konvergensi, sementara pengaturan learning rate yang adaptif dapat meningkatkan stabilitas pelatihan serta kemampuan generalisasi model.

Optimizers

Beberapa optimizer yang umum digunakan dalam pelatihan deep neural networks antara lain:

Momentum & Nesterov Accelerated Gradient (NAG)

Metode Momentum menambahkan komponen kecepatan pada proses Gradient Descent, sehingga pembaruan parameter dapat bergerak lebih cepat dan stabil, terutama saat melewati area dengan gradien kecil atau lembah yang sempit.
Nesterov Accelerated Gradient (NAG) merupakan pengembangan dari Momentum yang menghitung gradien pada posisi parameter yang telah diprediksi sebelumnya, sehingga sering kali menghasilkan konvergensi yang lebih cepat dan akurat.

AdaGrad

AdaGrad menerapkan learning rate adaptif untuk setiap parameter dengan cara memperkecil learning rate pada dimensi yang memiliki gradien besar. Pendekatan ini efektif untuk menangani fitur yang jarang muncul (sparse features). Namun, pada jaringan saraf yang dalam, learning rate cenderung mengecil terlalu cepat sehingga proses pelatihan dapat terhenti sebelum mencapai solusi optimal.

RMSProp

RMSProp merupakan perbaikan dari AdaGrad dengan hanya mempertimbangkan gradien terbaru melalui mekanisme exponential decay. Pendekatan ini membuat optimizer lebih stabil dan cocok digunakan pada pelatihan deep neural networks yang kompleks.

Adam & Nadam

Adam mengombinasikan konsep Momentum dan RMSProp dengan memperkirakan rata-rata (first moment) dan varians (second moment) gradien secara eksponensial. Dengan parameter default seperti learning rate 0.001, β₁ = 0.9, dan β₂ = 0.999, Adam sering memberikan performa yang baik tanpa memerlukan banyak penyesuaian parameter.
Nadam merupakan varian Adam yang mengintegrasikan Nesterov momentum, sehingga mampu mempercepat proses konvergensi pada beberapa kasus.

Learning Rate Scheduling

Penggunaan learning rate yang berubah selama proses pelatihan umumnya memberikan hasil yang lebih baik dibandingkan learning rate konstan. Strategi ini memungkinkan model untuk belajar secara agresif pada tahap awal dan melakukan penyesuaian yang lebih halus pada tahap akhir pelatihan. Beberapa pendekatan learning rate scheduling yang umum digunakan antara lain:

Exponential Decay, yaitu menurunkan learning rate secara bertahap mengikuti fungsi eksponensial.

Performance-based Scheduling, seperti ReduceLROnPlateau, yang menurunkan learning rate ketika performa model berhenti meningkat.

1cycle Policy, yang mengatur learning rate naik terlebih dahulu lalu turun secara bertahap, sehingga pelatihan menjadi lebih cepat dan generalisasi model meningkat.

In [ ]:
from tensorflow import keras

# Adam optimizer
adam_opt = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

# RMSProp optimizer
rms_opt = keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)

# SGD with Momentum + Nesterov
nag_opt = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=adam_opt,
              metrics=["accuracy"])

# Exponential learning rate decay (callback)
def exponential_decay_fn(epoch):
    return 0.01 * 0.1**(epoch / 20)

lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)

history = model.fit(X_train_scaled, y_train,
                    epochs=50,
                    validation_data=(X_valid_scaled, y_valid),
                    callbacks=[lr_scheduler])


6. Regularization: L1/L2, Dropout, MC Dropout, dan Max-Norm

Deep neural networks umumnya memiliki jumlah parameter yang sangat besar, sehingga rentan mengalami overfitting. Oleh karena itu, teknik regularisasi menjadi bagian penting dalam proses pelatihan model. Selain early stopping dan efek regularisasi implisit dari Batch Normalization, beberapa metode regularisasi utama dibahas pada bagian ini.

L1 dan L2 Regularization

L1 dan L2 regularization bekerja dengan menambahkan komponen penalti terhadap bobot ke dalam fungsi loss, sehingga kompleksitas model dapat dikendalikan.

L2 regularization (weight decay) berfungsi mengecilkan nilai bobot secara bertahap dan kontinu. Teknik ini membantu mencegah bobot menjadi terlalu besar, sehingga model menjadi lebih stabil dan memiliki kemampuan generalisasi yang lebih baik.

L1 regularization mendorong terbentuknya bobot nol pada banyak parameter, sehingga menghasilkan model yang lebih sparse. Selain mengurangi kompleksitas, pendekatan ini juga dapat meningkatkan interpretabilitas dan efisiensi komputasi.

Dropout

Dropout merupakan teknik regularisasi yang bekerja dengan cara menonaktifkan neuron secara acak selama proses pelatihan. Setiap neuron (kecuali neuron output) memiliki probabilitas tertentu p untuk diabaikan, sehingga output neuron tersebut menjadi nol.

Pendekatan ini memberikan beberapa keuntungan, antara lain:

memaksa jaringan untuk menyebarkan representasi informasi ke banyak neuron,

mengurangi ketergantungan berlebihan pada neuron tertentu,

serta bertindak seperti ensemble dari banyak subnet yang berbeda, sehingga mengurangi risiko overfitting.

Monte Carlo (MC) Dropout

MC Dropout merupakan pengembangan dari dropout standar dengan menerapkannya juga pada tahap inferensi. Dropout tetap diaktifkan, dan model dijalankan beberapa kali untuk input yang sama. Hasil prediksi kemudian dirata-ratakan untuk menghasilkan output akhir.

Teknik ini memiliki beberapa keunggulan, yaitu:

dapat meningkatkan akurasi prediksi,

memungkinkan estimasi ketidakpastian prediksi (predictive uncertainty),

sangat berguna pada aplikasi dengan tingkat risiko tinggi, seperti sistem medis atau keuangan.

Max-Norm Regularization

Max-Norm regularization membatasi norma vektor bobot agar tidak melebihi nilai ambang tertentu r. Jika norma bobot melebihi batas ini, bobot akan diskalakan kembali ke dalam rentang yang diizinkan.

Metode ini:

membantu mengontrol kompleksitas model,

menstabilkan proses pelatihan dengan mencegah bobot tumbuh terlalu besar,

dan sering dikombinasikan dengan dropout untuk menghasilkan model yang lebih robust.

In [ ]:
from tensorflow import keras
from functools import partial
import numpy as np

# L2-regularized dense layer helper
RegularizedDense = partial(
    keras.layers.Dense,
    activation="elu",
    kernel_initializer="he_normal",
    kernel_regularizer=keras.regularizers.l2(0.01)
)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    RegularizedDense(300),
    keras.layers.Dropout(rate=0.2),
    RegularizedDense(100),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax",
                       kernel_initializer="glorot_uniform",
                       kernel_constraint=keras.constraints.max_norm(1.0))
])

model.compile(loss="sparse_categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

history = model.fit(X_train_scaled, y_train,
                    epochs=30,
                    validation_data=(X_valid_scaled, y_valid))

# MC Dropout: rata-rata 100 prediksi dengan dropout aktif
y_probas = np.stack([
    model(X_test_scaled, training=True)  # penting: training=True
    for _ in range(100)
])
y_proba = y_probas.mean(axis=0)
y_pred = np.argmax(y_proba, axis=1)

7. Practical Default Configurations
Bab ini menutup pembahasan dengan beberapa konfigurasi default praktis yang dapat dijadikan titik awal dalam melatih deep neural networks.

General Deep Neural Networks
Untuk jaringan saraf dalam secara umum, konfigurasi yang sering bekerja dengan baik adalah:

He initialization dengan fungsi aktivasi ELU
Batch Normalization jika jaringan cukup dalam
Early stopping, dengan tambahan L2 regularization bila diperlukan
Optimizer berbasis momentum, seperti Momentum, RMSProp, atau Nadam
Learning rate schedule 1cycle untuk mempercepat konvergensi dan meningkatkan generalisasi
Self-Normalizing Dense Networks
Untuk arsitektur dense feedforward yang dirancang agar bersifat self-normalizing, konfigurasi yang direkomendasikan meliputi:

LeCun initialization dengan fungsi aktivasi SELU
Tanpa Batch Normalization
Alpha Dropout jika regularisasi tambahan diperlukan
Optimizer dan learning rate scheduling yang serupa dengan konfigurasi umum
Pendekatan ini menjaga mean dan standar deviasi aktivasi tetap stabil di seluruh jaringan.

Data-Efficient Training Strategies
Ketika data berlabel terbatas, sangat dianjurkan untuk memanfaatkan:

Transfer learning
Unsupervised atau self-supervised pretraining
Auxiliary tasks untuk membantu pembelajaran representasi
Strategi-strategi ini secara signifikan meningkatkan efisiensi pelatihan dan performa model.